# Downloading file

In [2]:
#Ensure we are using the right kernel
spark.version

'3.1.3'

In [3]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [9]:
directory = 'gs://msca-bdp-tweets/final_project/'
# file = '*.json'
# path = directory + file
path = directory

In [10]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

497.0 G  497.0 G  gs://msca-bdp-tweets/final_project



In [11]:
tweets_df = spark.read.json(path)

22/12/03 00:44:44 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
%%time

tweets_df.count()

CPU times: user 1.55 s, sys: 377 ms, total: 1.93 s
Wall time: 7min 23s


99992797

In [10]:
#These tweets are collected on the topics of education, schools, universities, learning, 
#knowledge sharing, etc., but only a fraction of them would be directly related to 
#either primary, secondary or higher education.

In [ ]:
#whether Twitter can be considered a credible source of information, which reflects 
#the emergence of important trends or topics in education.

In [11]:
#Remember, your key objective is to analyze the profiles of Twitterers (not the actual text messages of the Tweets)

#### Understanding the Data

In [12]:
tweets_df.limit(1)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_copyright,withheld_in_countries
null,Wed May 25 00:06:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529252559009067013,1529252559009067013,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,ChrChristensen,"{null, Tue May 24...","<a href=""http://t...",RT @ChrChristense...,1653437196863,false,"26 years ago, a g...","{false, Fri Sep 1...",null,null


In [12]:
tweets_df.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

In [13]:
tweets_df_en = tweets_df.filter(tweets_df.lang == 'en')

In [14]:
tweets_df_key = tweets_df_en.filter((lower(col('text')).contains("education"))\
                          | (lower(col('text')).contains("instruction"))\
                          | (lower(col('text')).contains("curriculum"))\
                          | (lower(col('text')).contains("school"))\
                          | (lower(col('text')).contains("learning"))\
                          | (lower(col('text')).contains("college"))\
                          | (lower(col('text')).contains("university"))\
                          | (lower(col('text')).contains("scholarship"))\
                          | (lower(col('text')).contains("tuition"))\
                          | (lower(col('text')).contains("k-12"))\
                          | (lower(col('text')).contains("high school"))\
                          | (lower(col('text')).contains("coeducation"))\
                          | (lower(col('text')).contains("self-education"))\
                          | (lower(col('text')).contains("course of study"))\
                          | (lower(col('text')).contains("homeschooling"))\
                          | (lower(col('text')).contains("grammar school"))\
                          | (lower(col('text')).contains("private education"))\
                          | (lower(col('text')).contains("public eduacation"))\
                          | (lower(col('text')).contains("online education"))\
                          | (lower(col('text')).contains("uniform"))\
                          | (lower(col('text')).contains("grade inflation"))\
                          | (lower(col('text')).contains("education for all"))\
                          | (lower(col('text')).contains("summer school")))


#https://relatedwords.org/relatedto/education

In [9]:
tweets_df_key.count()

22/12/02 23:36:35 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670022884528_0003_01_000032 on host: hub-msca-bdp-dphub-students-backup-sanchit-sw-chzt.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-02 23:36:35.012]Container killed on request. Exit code is 143
[2022-12-02 23:36:35.024]Container exited with a non-zero exit code 143. 
[2022-12-02 23:36:35.025]Killed by external signal
.
22/12/02 23:36:35 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 32 for reason Container from a bad node: container_1670022884528_0003_01_000032 on host: hub-msca-bdp-dphub-students-backup-sanchit-sw-chzt.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-02 23:36:35.012]Container killed on request. Exit code is 143
[2022-12-02 23:36:35.024]Container exited with a non-zero exit code 143. 
[2022-12-02 23:36:35.025]Killed by external signal
.
22/12/02 2

KeyboardInterrupt: 

In [15]:
tweets_df_key = tweets_df_key.limit(10000)
#10000 limited to first identify null values / etc before continuing analysis

In [16]:
from google.cloud import storage

In [17]:
bucket_read = 'msca-bdp-data-open'

In [18]:
bucket_write = 'msca-bdp-students-bucket'

In [19]:
def list_blobs(bucket_name, folder_name):
    """List all files in given COS directory."""    
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    for blob in blobs:
        print(blob.name + '\t' + str(blob.size))

In [ ]:
tweets_df_key.write.format("parquet").\
mode('overwrite').\
save('gs://' + bucket_write + '/shared_data/sanchit/tweets')

print('done')

done
